In [1]:
import perceval as pcvl
from perceval.rendering.circuit import SymbSkin, PhysSkin
from typing import Union
from perceval.components.unitary_components import Unitary, BS, PS, StateVector, BasicState
import numpy as np
from tqdm import tqdm
#from tensorboard import summary as summary_lib
import scipy

In [2]:
# Input: parameters of the discriminator, starting index of first node, name of discrim
# Output: discriminator
def create_discriminator(discriminator_params, start_mode, called):
    pses = []
    for i in range(6):
        ps = PS(phi=pcvl.P(called + f'{i}'))
        ps.assign({called + f'{i}':discriminator_params[i]})
        pses.append(ps)
        
    
    discriminator = pcvl.Circuit(4, name=called)
    discriminator.add(start_mode, pses[0])
    discriminator.add(start_mode+1, pses[1])
    discriminator.add(start_mode+2, pses[2])
    discriminator.add(start_mode+0, BS())
    discriminator.add(start_mode+2, BS())
    discriminator.add(start_mode, pses[3])
    discriminator.add(start_mode+2, pses[4])
    discriminator.add(start_mode, BS())
    discriminator.add(start_mode+2, BS())
    discriminator.add(start_mode+1, BS())
    discriminator.add(start_mode+1, pses[5])
    discriminator.add(start_mode+1, BS())
    
    return discriminator

# Input: discriminator and new parameters
# What it does: changes directly value of discriminator
def set_params_discriminator(discriminator, new_params):
    params = discriminator.get_parameters()
    for i in range(6):
        params[i].set_value(new_params[i])

In [3]:
def create_generator(generator_params, start_mode, called):
    quantum_circuit = pcvl.Circuit(4, name=called)
    
    pses = []
    for i in range(15):
        ps = PS(phi=pcvl.P(called + f'{i}'))
        ps.assign({called + f'{i}':generator_params[i]})
        pses.append(ps)
    
    quantum_circuit.add(start_mode + 1, pses[0])
    quantum_circuit.add(start_mode + 2, pses[1])
    quantum_circuit.add(start_mode + 3, pses[2])

    quantum_circuit.add(start_mode + 1, BS())
    quantum_circuit.add(start_mode + 1, pses[7])
    quantum_circuit.add(start_mode + 1, BS())

    # env1
    quantum_circuit.add(start_mode, BS())
    quantum_circuit.add(start_mode, pses[3])
    quantum_circuit.add(start_mode, BS())
    quantum_circuit.add(start_mode+1, pses[4])

    quantum_circuit.add(start_mode + 2, BS())
    quantum_circuit.add(start_mode + 2, pses[8])
    quantum_circuit.add(start_mode + 2, BS())

    # env2
    quantum_circuit.add(start_mode + 1, BS())
    quantum_circuit.add(start_mode + 1, pses[5])
    quantum_circuit.add(start_mode + 1, BS())
    quantum_circuit.add(start_mode + 2, pses[6])

    quantum_circuit.add(start_mode, pses[9])

    quantum_circuit.add(start_mode, BS())
    quantum_circuit.add(start_mode + 2, BS())


    quantum_circuit.add(start_mode, pses[10])
    quantum_circuit.add(start_mode + 2, pses[11])

    quantum_circuit.add(start_mode, BS())
    quantum_circuit.add(start_mode + 2, BS())

    quantum_circuit.add(start_mode, pses[12])
    quantum_circuit.add(start_mode + 1, pses[13])
    quantum_circuit.add(start_mode + 2, pses[14])
    
    return quantum_circuit
    
def set_params_generator(generator, new_params):
    params = generator.get_parameters()
    for i in range(15):
        params[i].set_value(new_params[i])    

In [4]:
## Initialize our circuit
DISCRIM_SIZE = 6
GEN_SIZE = 15

disc_init1 = np.random.randint(0,2,size=(DISCRIM_SIZE))
disc_init2 = np.random.randint(0,2,size=(DISCRIM_SIZE))
gen_init1 = np.random.randint(0,2,size=(GEN_SIZE))
gen_init2 = np.random.randint(0,2,size=(GEN_SIZE))
discrim1 = create_discriminator(disc_init1,0,'d_0')
discrim2 = create_discriminator(disc_init2,0,'d_1')
gen1 = create_generator(gen_init1,0,'g_0')
gen2 = create_generator(gen_init2,0,'g_1')

tdiscrim = pcvl.Circuit(8)
tdiscrim.add(0, discrim1)
tdiscrim.add(4, discrim2)

tgen = pcvl.Circuit(8)
tgen.add(0, gen1)
tgen.add(4, gen2)

In [5]:
##Backprop 

class Backprop():
    def __init__(self, gen, discrim):
        self.gen = gen
        self.discrim = discrim
        self.discrim_init = np.array([param.evalf() for param in discrim.get_parameters()])
        self.gen_init = np.array([param.evalf() for param in gen.get_parameters()])
        self.simulator = pcvl.Simulator(pcvl.NaiveBackend())
        self.tau_sv = StateVector("|1,0,0,0,0,1,0,0>") + StateVector("|0,1,0,0,0,0,1,0>") + StateVector("|0,0,1,0,0,0,0,1>") + StateVector("|0,0,0,1,1,0,0,0>")
        self.simulator.set_circuit(self.gen)
        self.initial_sv = StateVector("|1,0,0,0,1,0,0,0>") + StateVector("|0,1,0,0,0,1,0,0>") + StateVector("|0,0,1,0,0,0,1,0>") + StateVector("|0,0,0,1,0,0,0,1>")
        
    def get_density_matrix(self,sv):
        basic_states_8 = [BasicState("|1,0,0,0,1,0,0,0>"), # |00>
                        BasicState("|1,0,0,0,0,1,0,0>"), # |01>
                        BasicState("|1,0,0,0,0,0,1,0>"), # |02>
                        BasicState("|1,0,0,0,0,0,0,1>"), # |03>
                        BasicState("|0,1,0,0,1,0,0,0>"), # |10>
                        BasicState("|0,1,0,0,0,1,0,0>"), # |11>
                        BasicState("|0,1,0,0,0,0,1,0>"), # |12>
                        BasicState("|0,1,0,0,0,0,0,1>"), # |13>
                        BasicState("|0,0,1,0,1,0,0,0>"), # |20>
                        BasicState("|0,0,1,0,0,1,0,0>"), # |21>
                        BasicState("|0,0,1,0,0,0,1,0>"), # |22>
                        BasicState("|0,0,1,0,0,0,0,1>"), # |23>
                        BasicState("|0,0,0,1,1,0,0,0>"), # |30>
                        BasicState("|0,0,0,1,0,1,0,0>"), # |31>
                        BasicState("|0,0,0,1,0,0,1,0>"), # |32>
                        BasicState("|0,0,0,1,0,0,0,1>"), # |33>
                        ]
        sv_amplitudes = np.array([])
        for state in basic_states_8:
            sv_amplitudes = np.append(sv_amplitudes, sv[state])
        sv_amplitudes_bra = np.conjugate(sv_amplitudes.T)
        rho = np.outer(sv_amplitudes, sv_amplitudes_bra)
        return rho


    def make_mats(self):    
        tau = self.get_density_matrix(self.tau_sv)
        return tau

    def D(self,psi_G):
        rho = self.get_density_matrix(psi_G)
        # print(rho)
        p_s = pcvl.Matrix(discrim1.U)
        p_i = pcvl.Matrix(discrim2.U)
        M_hat = np.kron(p_s, p_i)

        expectation_value_rho = np.trace(M_hat @ rho)
        expectation_value_tau = np.trace(M_hat @ self.make_mats())
        return np.abs(expectation_value_rho - expectation_value_tau)            
    
    def prop_gen(self, params_list):
        dist_i = self.D(self.simulator.evolve(self.initial_sv))

        for i,param in enumerate(self.gen.get_parameters()):
            param.set_value(params_list[i])
        self.simulator.set_circuit(self.gen)
        dist_f = self.D(self.simulator.evolve(self.initial_sv))
        #print(dist_i,dist_f)
        print(f"G:{dist_f}")
        return dist_f
    
    def prop_discrim(self, params_list):
        dist_i = self.D(self.simulator.evolve(self.initial_sv))

        for i,param in enumerate(self.discrim.get_parameters()):
            param.set_value(params_list[i])
        self.simulator.set_circuit(self.discrim)
        dist_f = self.D(self.simulator.evolve(self.initial_sv))
        #print(dist_i,dist_f)
        print(f"D:{dist_f}")
        return -dist_f
    
    def run_gen_optim(self):
        options = {'maxiter': 5}
        out=scipy.optimize.minimize(self.prop_gen, self.gen_init, options=options)
        print(f"G:{out}")
        
    def run_discrim_optim(self):
        options = {'maxiter': 5}
        out=scipy.optimize.minimize(self.prop_discrim, self.discrim_init,options=options)
        print(f"D:{out}")
    
    def mog(self, epochs):
        for epoch in tqdm(range(epochs)):
            self.run_gen_optim()
            self.run_discrim_optim()
            self.run_gen_optim()
        return self.gen

            
Back = Backprop(tgen,tdiscrim)

gen = Back.mog(1)


simulator = pcvl.Simulator(pcvl.NaiveBackend())
simulator.set_circuit(gen)
print(simulator.evolve(StateVector("|1,0,0,0,1,0,0,0>") + StateVector("|0,1,0,0,0,1,0,0>") + StateVector("|0,0,1,0,0,0,1,0>") + StateVector("|0,0,0,1,0,0,0,1>")))

  0%|                                                     | 0/1 [00:00<?, ?it/s]

G:0.681740814106979
G:0.681740813657718
G:0.6817408145623288
G:0.6817408137999821
G:0.6817408133243729
G:0.6817408114169566
G:0.6817408138542297
G:0.6817408137924196
G:0.681740815377285
G:0.6817408123005425
G:0.6817408152028425
G:0.6817408144872542
G:0.6817408121379597
G:0.6817408163173133
G:0.6817408152028426
G:0.6817408122769165
G:0.681740813657718
G:0.6817408145623286
G:0.6817408137999821
G:0.6817408142724711
G:0.68174081226189
G:0.6817408153303275
G:0.6817408144117758
G:0.6817408152046207
G:0.6817408140403362
G:0.6817408136398876
G:0.681740812174405
G:0.6817408155448293
G:0.6817408150899734
G:0.6817408136398876
G:0.6817408134066388
G:0.47122700179237775
G:0.4712270016971814
G:0.4712270020405399
G:0.4712270014947165
G:0.47122700043440846
G:0.47122699903429655
G:0.471227001083705
G:0.47122700168055437
G:0.47122700252713323
G:0.4712269993223524
G:0.4712270033881189
G:0.4712270025911187
G:0.47122699960968695
G:0.4712270041697928
G:0.4712270034411362
G:0.47122699957663794
G:0.4712270016

G:0.0022866806791027824
G:  message: Maximum number of iterations has been exceeded.
  success: False
   status: 1
      fun: 0.0022866773383830715
        x: [ 9.264e-02 -1.578e-01 ...  1.128e+00  1.137e+00]
      nit: 5
      jac: [ 1.626e-01 -8.904e-02 ...  8.740e-03  2.242e-01]
 hess_inv: [[ 9.367e-01  2.858e-02 ... -5.016e-03 -8.733e-02]
            [ 2.858e-02  9.690e-01 ... -2.113e-02  2.767e-02]
            ...
            [-5.016e-03 -2.113e-02 ...  9.593e-01  1.197e-03]
            [-8.733e-02  2.767e-02 ...  1.197e-03  8.654e-01]]
     nfev: 372
     njev: 12
D:0.7271139493232819
D:0.7271139497741744
D:0.7271139518294032
D:0.7271139479877707
D:0.7271139451464832
D:0.7271139507326235
D:0.7271139451904609
D:0.7271139523270919
D:0.7271139472122543
D:0.7271139509597626
D:0.7271139443424481
D:0.7271139544857934
D:0.7271139518069399
D:1.1911373243886663
D:1.1911373239594814
D:1.1911373277299708
D:1.1911373228397095
D:1.1911373194981896
D:1.1911373239191638
D:1.1911373214218144
D:1

100%|████████████████████████████████████████████| 1/1 [06:36<00:00, 396.80s/it]

G:1.2741964874239509
G:  message: Maximum number of iterations has been exceeded.
  success: False
   status: 1
      fun: 1.2741964880117085
        x: [ 2.874e-02 -3.537e-01 ...  1.180e+00  4.989e-01]
      nit: 5
      jac: [-3.188e-02 -5.539e-02 ... -6.687e-02 -3.944e-02]
 hess_inv: [[ 9.416e-01 -1.777e-03 ... -6.353e-04 -4.668e-02]
            [-1.777e-03  1.080e+00 ... -7.596e-02  6.791e-02]
            ...
            [-6.353e-04 -7.596e-02 ...  1.076e+00 -9.005e-02]
            [-4.668e-02  6.791e-02 ... -9.005e-02  1.097e+00]]
     nfev: 248
     njev: 8


(0.197378-0.200836*I)*|0,1,0,0,0,0,0,1>+(0.366279+0.068167*I)*|0,0,0,1,1,0,0,0>+(0.137784+0.182297*I)*|0,0,1,0,0,0,1,0>+(-0.032265+0.111309*I)*|0,0,0,1,0,0,1,0>+(-0.229293-0.202016*I)*|0,0,1,0,0,1,0,0>+(-0.13237-0.260122*I)*|0,0,0,1,0,0,0,1>+(-0.161502+0.089359*I)*|0,0,1,0,1,0,0,0>+(-0.108204-0.059016*I)*|1,0,0,0,0,0,0,1>+(0.239695-0.214815*I)*|0,1,0,0,0,0,1,0>+(-0.212329-0.158891*I)*|0,0,1,0,0,0,0,1>+(-0.160475-0.04444*I)*|0,1,0,0,0,1,0,0>+(0.084077-0.179754*I)*|0,1,0,0,1,0,0,0>+(0.053847+0.098373*I)*|0,0,0,1,0,1,0,0>+(0.186643-0.214285*I)*|1,0,0,0,0,0,1,0>+(0.310694-0.140642*I)*|1,0,0,0,0,1,0,0>+(-0.042635+0.189544*I)*|1,0,0,0,1,0,0,0>
